In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r'E:\Network failure classification model using network digital twin\training-data_a.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Network failure classification model using network digital twin\\training-data_a.csv'

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df_c = pd.read_csv(r'E:\Network failure classification model using network digital twin\training-data_c.csv')

In [ ]:
df_c.head()

In [ ]:
del df_c['time']
del df_c['source_name']
del df_c['upfx7_per-core-stats.per-core-stat.user_value']

In [ ]:
df_c.head()

In [ ]:
df_c.tail()

In [ ]:
df_c['y_true(fc)'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
label = le.fit_transform(df_c['y_true(fc)'])
df_c["y_true(fc)"] = label

In [ ]:
df_c["y_true(fc)"].value_counts()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
# Silence warnings
import warnings
warnings.filterwarnings('ignore')

In [24]:
X = df_c.drop('y_true(fc)', axis=1)
y = df_c["y_true(fc)"]

In [33]:
y

0      10
1      15
2       3
3      10
4      10
       ..
455    10
456    10
457    10
458    15
459     6
Name: y_true(fc), Length: 460, dtype: int32

In [26]:
kfold = StratifiedKFold(n_splits=5)
from sklearn.model_selection import cross_val_score

def classification_model(model):
    scores = cross_val_score(model, X, y, cv=kfold)
    return scores.mean()

In [27]:
classification_model(XGBClassifier())

0.9173913043478261

In [28]:
classification_model(XGBClassifier(booster='gblinear'))

0.6934782608695652

In [29]:
classification_model(XGBClassifier(booster='dart', one_drop=True))

0.9217391304347826

In [30]:
classification_model(RandomForestClassifier(random_state=2))

0.8282608695652174

In [31]:
classification_model(LogisticRegression(max_iter=10000))

0.6891304347826087

In [32]:
classification_model(XGBClassifier(n_estimators=800, max_depth=4, colsample_bylevel=0.8))

0.923913043478261

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [36]:
def y_pred(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_pred, y_test)
    print(score)
    return y_pred

In [37]:
y_pred_gbtree = y_pred(XGBClassifier())

0.8695652173913043


In [38]:
y_pred_dart = y_pred(XGBClassifier(booster='dart', one_drop=True))

0.8782608695652174


In [39]:
y_pred_forest = y_pred(RandomForestClassifier(random_state=2))

0.8173913043478261


In [40]:
y_pred_logistic = y_pred(LogisticRegression(max_iter=10000))

0.6782608695652174


In [41]:
y_pred_xgb = y_pred(XGBClassifier(max_depth=2, n_estimators=500, learning_rate=0.1))

0.8782608695652174


In [42]:
df_pred = pd.DataFrame(data= np.c_[y_pred_gbtree, y_pred_dart, y_pred_forest, y_pred_logistic, y_pred_xgb],  columns=['gbtree', 'dart', 'forest', 'logistic', 'xgb'])

In [43]:
df_pred.corr()

,gbtree,dart,forest,logistic,xgb
gbtree,1.000000,0.983850,0.699115,0.204184,0.970680
dart,0.983850,1.000000,0.722418,0.177248,0.985544
forest,0.699115,0.722418,1.000000,0.218166,0.722987
logistic,0.204184,0.177248,0.218166,1.000000,0.208713
xgb,0.970680,0.985544,0.722987,0.208713,1.000000


In [44]:
estimators = []
logistic_model = LogisticRegression(max_iter=10000)
estimators.append(('logistic', logistic_model))
xgb_model = XGBClassifier(max_depth=2, n_estimators=500, learning_rate=0.1)
estimators.append(('xgb', xgb_model))
rf_model = RandomForestClassifier(random_state=2)
estimators.append(('rf', rf_model))
ensemble = VotingClassifier(estimators)
scores = cross_val_score(ensemble, X, y, cv=kfold)
print(scores.mean())

0.8565217391304347


In [45]:
base_models = []
base_models.append(('lr', LogisticRegression()))
base_models.append(('xgb', XGBClassifier()))
base_models.append(('rf', RandomForestClassifier(random_state=2)))
# define meta learner model
meta_model = LogisticRegression()
# define the stacking ensemble
clf = StackingClassifier(estimators=base_models, final_estimator=meta_model)
scores = cross_val_score(clf, X, y, cv=kfold)
print(scores.mean())

0.9065217391304348


In [46]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [49]:
validation_size = 0.2

In [50]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=validation_size, random_state=7)

In [51]:
scoring = 'accuracy'

In [52]:
num_folds = 10
seed = 7

In [53]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))


In [55]:
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.236486 (0.085680)
LDA: 0.611637 (0.068639)
KNN: nan (nan)
CART: 0.864489 (0.066938)


In [57]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

DecisionTreeClassifier()

In [60]:
rescaledValidationX = X_validation
predictions = model.predict(rescaledValidationX)
print(accuracy_score(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.9021739130434783
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         2
           5       0.40      0.67      0.50         3
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       0.98      0.98      0.98        56
          11       0.00      0.00      0.00         2
          12       0.75      0.75      0.75         4
          13       0.50      0.50      0.50         2
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         2

    accuracy                           0.90        92
   macr

In [61]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('NN', MLPClassifier()))
models.append(('AB', AdaBoostClassifier()))
models.append(('GBM', GradientBoostingClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('ET', ExtraTreesClassifier()))

In [63]:
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.241967 (0.051528)
LDA: 0.572297 (0.129611)
KNN: nan (nan)
CART: 0.885736 (0.059881)
NB: 0.155030 (0.046012)
SVM: 0.690090 (0.067934)
NN: 0.486336 (0.066020)
AB: 0.774474 (0.046744)
GBM: 0.869670 (0.039516)
RF: 0.820571 (0.079160)
ET: 0.823273 (0.044738)


In [67]:
from keras.wrappers.scikit_learn import KerasClassifier

In [68]:
EnableDLModelsFlag = 1
if EnableDLModelsFlag == 1 :   
    # Function to create model, required for KerasClassifier
    def create_model(neurons=12, activation='relu', learn_rate = 0.01, momentum=0):
        # create model
        model = Sequential()
        model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation=activation))
        model.add(Dense(32, activation=activation))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        optimizer = SGD(lr=learn_rate, momentum=momentum)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model    
    models.append(('DNN', KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0)))

In [69]:
# spot check the algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
#Neural Network
models.append(('NN', MLPClassifier()))
# #Ensable Models 
# Boosting methods
models.append(('AB', AdaBoostClassifier()))
models.append(('GBM', GradientBoostingClassifier()))
# Bagging methods
models.append(('RF', RandomForestClassifier()))
models.append(('ET', ExtraTreesClassifier()))

In [81]:
base_models = []
base_models.append(('lr', LogisticRegression()))
base_models.append(('xgb', XGBClassifier()))
base_models.append(('rf', RandomForestClassifier(random_state=2)))
meta_model = LogisticRegression()
clf = StackingClassifier(estimators=base_models, final_estimator=meta_model)
scores = cross_val_score(clf, X, y, cv=kfold)
print(scores.mean())

0.9021739130434783


In [34]:
import pandas as pd 

In [35]:
dfc = pd.read_csv(r'C:\Users\Jazzy RIG\OneDrive - Double Queue\Machine learning Comps\Network failure classification model using network digital twin\training-data_c.csv')

In [36]:
dfc.head()

,time,source_name,y_true(fc),amfx1_ens3_oper-status,amfx1_ens3_phys-address,amfx1_ens3_speed,amfx1_ens3_statistics.in-octets,amfx1_ens3_statistics.in-unicast-pkts,amfx1_ens3_statistics.out-octets,amfx1_ens3_statistics.out-unicast-pkts,...,upfx7_memory-stats.free-percent_value,upfx7_memory-stats.memory-status_value,upfx7_memory-stats.used-number_value,upfx7_memory-stats.used-percent_value,upfx7_per-core-stats.per-core-stat.idle_value,upfx7_per-core-stats.per-core-stat.io-wait_value,upfx7_per-core-stats.per-core-stat.nice_value,upfx7_per-core-stats.per-core-stat.sirq_value,upfx7_per-core-stats.per-core-stat.system_value,upfx7_per-core-stats.per-core-stat.user_value
0,20210128175112-4_20210128175200,network-5gc-c,normal,0,0,0,1,1,1,1,...,8,0,-148496,-8,-0.095,0.18,0,0.0,0.045,-4.500000e-02
1,20210128175529-6_20210128175610,network-5gc-c,udmx1_vcpu-overload-start,0,0,0,1,1,1,1,...,0,0,-24,0,-0.095,0.02,0,0.0,0.010,8.000000e-02
2,20210128175945-8_20210128180030,network-5gc-c,amfx1_memory-stress-start,0,0,0,1,1,1,1,...,-1,0,4092,1,-0.100,0.00,0,0.0,0.100,4.160000e-17
3,20210128180403-10_20210128180450,network-5gc-c,normal,0,0,0,1,1,1,1,...,0,0,244,0,0.350,-0.02,0,0.0,-0.030,-3.100000e-01
4,20210128180817-12_20210128180900,network-5gc-c,normal,0,0,0,1,1,1,1,...,0,0,160,0,-0.130,0.00,0,0.0,0.140,-5.000000e-03


In [37]:
dfc.describe()

,amfx1_ens3_oper-status,amfx1_ens3_phys-address,amfx1_ens3_speed,amfx1_ens3_statistics.in-octets,amfx1_ens3_statistics.in-unicast-pkts,amfx1_ens3_statistics.out-octets,amfx1_ens3_statistics.out-unicast-pkts,amfx1_ens3_oper-status_value,amfx1_ens3_phys-address_value,amfx1_ens3_speed_value,...,upfx7_memory-stats.free-percent_value,upfx7_memory-stats.memory-status_value,upfx7_memory-stats.used-number_value,upfx7_memory-stats.used-percent_value,upfx7_per-core-stats.per-core-stat.idle_value,upfx7_per-core-stats.per-core-stat.io-wait_value,upfx7_per-core-stats.per-core-stat.nice_value,upfx7_per-core-stats.per-core-stat.sirq_value,upfx7_per-core-stats.per-core-stat.system_value,upfx7_per-core-stats.per-core-stat.user_value
count,460.0,460.0,460.0,460.0,460.0,460.0,460.0,460.0,460.0,460.0,...,460.000000,460.0,460.000000,460.000000,460.000000,460.000000,460.0,460.000000,460.000000,460.000000
mean,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,-0.160870,0.0,3692.669565,0.160870,0.030533,-0.030837,0.0,0.001826,0.003174,0.004435
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.024094,0.0,146765.176059,7.024094,0.913984,0.899999,0.0,0.036772,0.117921,0.190960
min,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,-13.000000,0.0,-250672.000000,-13.000000,-1.415000,-18.475000,0.0,-0.110000,-0.420000,-0.600000
25%,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,-1.000000,0.0,-12478.000000,-1.000000,-0.140000,-0.070000,0.0,0.000000,-0.065000,-0.106250
50%,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.000000,0.0,158.000000,0.000000,-0.005000,0.000000,0.0,0.000000,0.000000,0.000000
75%,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.000000,0.0,21523.000000,1.000000,0.145000,0.060000,0.0,0.000000,0.080000,0.131250
max,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,13.000000,0.0,254108.000000,13.000000,18.770000,2.140000,0.0,0.200000,0.400000,0.710000


In [38]:
dfc.dtypes

time                                                 object
source_name                                          object
y_true(fc)                                           object
amfx1_ens3_oper-status                                int64
amfx1_ens3_phys-address                               int64
                                                     ...   
upfx7_per-core-stats.per-core-stat.io-wait_value    float64
upfx7_per-core-stats.per-core-stat.nice_value         int64
upfx7_per-core-stats.per-core-stat.sirq_value       float64
upfx7_per-core-stats.per-core-stat.system_value     float64
upfx7_per-core-stats.per-core-stat.user_value       float64
Length: 4121, dtype: object

In [39]:
del dfc['time']
del dfc['source_name']

In [40]:
dfc.head(1)

,y_true(fc),amfx1_ens3_oper-status,amfx1_ens3_phys-address,amfx1_ens3_speed,amfx1_ens3_statistics.in-octets,amfx1_ens3_statistics.in-unicast-pkts,amfx1_ens3_statistics.out-octets,amfx1_ens3_statistics.out-unicast-pkts,amfx1_ens3_oper-status_value,amfx1_ens3_phys-address_value,...,upfx7_memory-stats.free-percent_value,upfx7_memory-stats.memory-status_value,upfx7_memory-stats.used-number_value,upfx7_memory-stats.used-percent_value,upfx7_per-core-stats.per-core-stat.idle_value,upfx7_per-core-stats.per-core-stat.io-wait_value,upfx7_per-core-stats.per-core-stat.nice_value,upfx7_per-core-stats.per-core-stat.sirq_value,upfx7_per-core-stats.per-core-stat.system_value,upfx7_per-core-stats.per-core-stat.user_value
0,normal,0,0,0,1,1,1,1,0,0,...,8,0,-148496,-8,-0.095,0.18,0,0.0,0.045,-0.045


In [41]:
dfc.dtypes

y_true(fc)                                           object
amfx1_ens3_oper-status                                int64
amfx1_ens3_phys-address                               int64
amfx1_ens3_speed                                      int64
amfx1_ens3_statistics.in-octets                       int64
                                                     ...   
upfx7_per-core-stats.per-core-stat.io-wait_value    float64
upfx7_per-core-stats.per-core-stat.nice_value         int64
upfx7_per-core-stats.per-core-stat.sirq_value       float64
upfx7_per-core-stats.per-core-stat.system_value     float64
upfx7_per-core-stats.per-core-stat.user_value       float64
Length: 4119, dtype: object

In [42]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(dfc['y_true(fc)'])
dfc['y_true(fc)'] = label

In [53]:
 import sklearn
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report, roc_curve

In [56]:
X = dfc.drop('y_true(fc)', axis=1)
y = dfc['y_true(fc)']

In [57]:
from xgboost import XGBClassifier

# Import accuracy_score
from sklearn.metrics import accuracy_score

# Import train_test_split
from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)


In [58]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=2)

# Fit xg_reg to training set
model.fit(X_train, y_train)

# Predict labels of test set, y_pred
y_pred = model.predict(X_test)

score = accuracy_score(y_pred, y_test)

print('Score: ' + str(score))

Score: 0.8695652173913043


In [59]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       1.00      1.00      1.00         3
           2       1.00      0.33      0.50         3
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         3
           5       0.00      0.00      0.00         3
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         3
          10       0.87      0.97      0.92        75
          11       0.00      0.00      0.00         3
          12       1.00      0.80      0.89         5
          14       1.00      1.00      1.00         2
          15       0.75      1.00      0.86         3

    accuracy                           0.87       115
   macro avg       0.77      0.74      0.74       115
weighted avg       0.82   

In [62]:
from sklearn.metrics import recall_score, 

In [65]:
def xgb_clf(model, nrows):
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

    # Fit xg_reg to training set
    model.fit(X_train, y_train)

    # Predict labels of test set, y_pred
    y_pred = model.predict(X_test)

    score = recall_score(y_test, y_pred, pos_label='positive', average='micro')
    
    print(confusion_matrix(y_test, y_pred))
    
    print(classification_report(y_test, y_pred))
        
    return score

In [66]:
xgb_clf(XGBClassifier(random_state=2), nrows=800)


[[ 0  0  0  0  0  1  0  0  0  0  3  0  0  0  0]
 [ 0  3  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  3  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  3  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  3  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  2  0  0  0  0 73  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  3  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  4  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  3]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       1.00      1.00      1.00         3
           2       1.00      0.33      0.50         3
           3       1.00      1.00      1.00         2
        

0.8695652173913043

In [69]:
X_all = dfc.iloc[:,1:]
y_all = dfc.iloc[:,0]

In [70]:
X_all

,amfx1_ens3_oper-status,amfx1_ens3_phys-address,amfx1_ens3_speed,amfx1_ens3_statistics.in-octets,amfx1_ens3_statistics.in-unicast-pkts,amfx1_ens3_statistics.out-octets,amfx1_ens3_statistics.out-unicast-pkts,amfx1_ens3_oper-status_value,amfx1_ens3_phys-address_value,amfx1_ens3_speed_value,...,upfx7_memory-stats.free-percent_value,upfx7_memory-stats.memory-status_value,upfx7_memory-stats.used-number_value,upfx7_memory-stats.used-percent_value,upfx7_per-core-stats.per-core-stat.idle_value,upfx7_per-core-stats.per-core-stat.io-wait_value,upfx7_per-core-stats.per-core-stat.nice_value,upfx7_per-core-stats.per-core-stat.sirq_value,upfx7_per-core-stats.per-core-stat.system_value,upfx7_per-core-stats.per-core-stat.user_value
0,0,0,0,1,1,1,1,0,0,0,...,8,0,-148496,-8,-0.095,0.180,0,0.0,0.045,-4.500000e-02
1,0,0,0,1,1,1,1,0,0,0,...,0,0,-24,0,-0.095,0.020,0,0.0,0.010,8.000000e-02
2,0,0,0,1,1,1,1,0,0,0,...,-1,0,4092,1,-0.100,0.000,0,0.0,0.100,4.160000e-17
3,0,0,0,1,1,1,1,0,0,0,...,0,0,244,0,0.350,-0.020,0,0.0,-0.030,-3.100000e-01
4,0,0,0,1,1,1,1,0,0,0,...,0,0,160,0,-0.130,0.000,0,0.0,0.140,-5.000000e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,0,0,0,1,1,1,1,0,0,0,...,-11,0,234920,11,-0.070,0.000,0,0.0,-0.200,1.600000e-01
456,0,0,0,1,1,1,1,0,0,0,...,0,0,92,0,-0.140,0.270,0,0.0,-0.180,9.500000e-02
457,0,0,0,1,1,1,1,0,0,0,...,12,0,-242900,-12,-0.220,0.200,0,0.0,-0.190,2.100000e-01
458,0,0,0,1,1,1,1,0,0,0,...,12,0,-248856,-12,-0.140,0.015,0,0.0,0.075,5.000000e-02


In [71]:
y_all

0      10
1      15
2       3
3      10
4      10
       ..
455    10
456    10
457    10
458    15
459     6
Name: y_true(fc), Length: 460, dtype: int32

In [74]:
X_short = X.iloc[:74, :]
y_short = y.iloc[:74]

In [77]:
def final_model(X, y, model):
    model.fit(X, y)
    y_pred = model.predict(X_all)
    score = recall_score(y_all, y_pred, pos_label='positive', average='micro')
    print(score)
    print(confusion_matrix(y_all, y_pred))
    print(classification_report(y_all, y_pred))

In [78]:
final_model(X_short, y_short, XGBClassifier(max_depth=2, colsample_by_node=0.5, random_state=2))


[17:32:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "colsample_by_node" } are not used.

0.7695652173913043
[[  1   2   0   0   0   0   0   1   0   0   6   0   0   0   0   0]
 [  0   2   3   0   0   0   0   0   0   0   3   2   0   0   0   0]
 [  0   0   4   0   0   0   0   0   0   0   4   0   1   1   0   0]
 [  0   0   0   5   0   0   0   0   0   0   5   0   0   0   0   0]
 [  0   0   0   0   1   0   0   0   0   0   9   0   0   0   0   0]
 [  0   0   2   0   0   1   0   0   0   0   6   0   1   0   0   0]
 [  0   2   0   0   0   0   1   0   0   0   6   0   1   0   0   0]
 [  0   2   0   0   0   0   0   1   1   0   5   0   0   1   0   0]
 [  0   0   0   0   0   0   0   0   8   0   2   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   7   3   0   0   0   0   0]
 [  0   2   3   0   0   1   0   2   2   0 297   0   2   1   0   0]
 [  0   1   0   0   0   0   0   0   0

In [79]:
final_model(X, y, XGBClassifier(max_depth=2, colsample_bynode=0.5, scale_pos_weight=10, random_state=2))


[17:32:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

1.0
[[ 10   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  10   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  10   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  10   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  10   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  10   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  10   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  10   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  10   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  10   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 310   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0  10   0

In [82]:
weight = int(5050/37)


In [83]:
final_model(X_all, y_all, XGBClassifier(max_depth=2, colsample_bynode=0.5, scale_pos_weight=weight, random_state=2))


[17:33:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

1.0
[[ 10   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  10   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  10   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  10   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  10   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  10   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  10   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  10   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  10   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  10   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 310   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0  10   0

In [85]:
from sklearn.model_selection import train_test_split

x = dfc.drop('y_true(fc)', axis=1)
y = dfc['y_true(fc)']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=0)


In [92]:
y_predicted = model.predict(x_test) > 0.5

3/3 [==============================] - 0s 1ms/step


In [93]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.02      1.00      0.04         2
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00        62
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         2

    accuracy                           0.02        92
   macro avg       0.00   

In [94]:
df = pd.read_csv(r'E:\Network failure classification model using network digital twin\training-data_a.csv')

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3645 entries, 0 to 3644
Columns: 4121 entries, time to upfx7_per-core-stats.per-core-stat.user_value
dtypes: float64(215), int64(3903), object(3)
memory usage: 114.6+ MB


In [96]:
df.head(1)

,time,source_name,y_true(fc),amfx1_ens3_oper-status,amfx1_ens3_phys-address,amfx1_ens3_speed,amfx1_ens3_statistics.in-octets,amfx1_ens3_statistics.in-unicast-pkts,amfx1_ens3_statistics.out-octets,amfx1_ens3_statistics.out-unicast-pkts,...,upfx7_memory-stats.free-percent_value,upfx7_memory-stats.memory-status_value,upfx7_memory-stats.used-number_value,upfx7_memory-stats.used-percent_value,upfx7_per-core-stats.per-core-stat.idle_value,upfx7_per-core-stats.per-core-stat.io-wait_value,upfx7_per-core-stats.per-core-stat.nice_value,upfx7_per-core-stats.per-core-stat.sirq_value,upfx7_per-core-stats.per-core-stat.system_value,upfx7_per-core-stats.per-core-stat.user_value
0,20210128175112-4_20210128175200,network-5gc-a,udmx1_vcpu-overload-start,0,0,0,1,1,1,1,...,0,0,-3308,0,-0.22,0.03,0,0.0,-0.03,0.24


In [97]:
del df['time']
del df['source_name']

In [99]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df['y_true(fc)'])
df['y_true(fc)'] = label

In [100]:
df.head(1)

,y_true(fc),amfx1_ens3_oper-status,amfx1_ens3_phys-address,amfx1_ens3_speed,amfx1_ens3_statistics.in-octets,amfx1_ens3_statistics.in-unicast-pkts,amfx1_ens3_statistics.out-octets,amfx1_ens3_statistics.out-unicast-pkts,amfx1_ens3_oper-status_value,amfx1_ens3_phys-address_value,...,upfx7_memory-stats.free-percent_value,upfx7_memory-stats.memory-status_value,upfx7_memory-stats.used-number_value,upfx7_memory-stats.used-percent_value,upfx7_per-core-stats.per-core-stat.idle_value,upfx7_per-core-stats.per-core-stat.io-wait_value,upfx7_per-core-stats.per-core-stat.nice_value,upfx7_per-core-stats.per-core-stat.sirq_value,upfx7_per-core-stats.per-core-stat.system_value,upfx7_per-core-stats.per-core-stat.user_value
0,15,0,0,0,1,1,1,1,0,0,...,0,0,-3308,0,-0.22,0.03,0,0.0,-0.03,0.24


In [101]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
# Silence warnings
import warnings
warnings.filterwarnings('ignore')

In [103]:
X = df.drop('y_true(fc)', axis=1)
y = df['y_true(fc)']

In [104]:
kfold = StratifiedKFold(n_splits=5)
from sklearn.model_selection import cross_val_score

def classification_model(model):
    # Obtain scores of cross-validation using 5 splits
    scores = cross_val_score(model, X, y, cv=kfold)

    # Return mean score
    return scores.mean()

In [105]:
classification_model(XGBClassifier())


0.9739368998628258

In [106]:
classification_model(XGBClassifier(n_estimators=800, max_depth=4, colsample_bylevel=0.8))


0.9731138545953361

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)


In [110]:
def y_pred(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_pred, y_test)
    print(score)
    return y_pred

In [111]:
y_pred_gbtree = y_pred(XGBClassifier())


0.9692982456140351


In [113]:
y_pred_forest = y_pred(RandomForestClassifier(random_state=2))


0.9265350877192983


In [114]:
y_pred_logistic = y_pred(LogisticRegression(max_iter=10000))


0.6140350877192983


In [115]:
y_pred_xgb = y_pred(XGBClassifier(max_depth=2, n_estimators=500, learning_rate=0.1))

0.9703947368421053


In [120]:
df_pred = pd.DataFrame(data= np.c_[y_pred_gbtree, y_pred_forest, y_pred_logistic, y_pred_xgb],  columns=['gbtree', 'forest', 'logistic', 'xgb'])

In [123]:
df_pred.corr()

,gbtree,forest,logistic,xgb
gbtree,1.000000,0.948955,0.035535,0.991014
forest,0.948955,1.000000,0.030469,0.949369
logistic,0.035535,0.030469,1.000000,0.022409
xgb,0.991014,0.949369,0.022409,1.000000


In [154]:
df.isna().sum()

y_true(fc)                                          0
amfx1_ens3_oper-status                              0
amfx1_ens3_phys-address                             0
amfx1_ens3_speed                                    0
amfx1_ens3_statistics.in-octets                     0
                                                   ..
upfx7_per-core-stats.per-core-stat.io-wait_value    0
upfx7_per-core-stats.per-core-stat.nice_value       0
upfx7_per-core-stats.per-core-stat.sirq_value       0
upfx7_per-core-stats.per-core-stat.system_value     0
upfx7_per-core-stats.per-core-stat.user_value       0
Length: 4119, dtype: int64

In [155]:
df.head(1)

,y_true(fc),amfx1_ens3_oper-status,amfx1_ens3_phys-address,amfx1_ens3_speed,amfx1_ens3_statistics.in-octets,amfx1_ens3_statistics.in-unicast-pkts,amfx1_ens3_statistics.out-octets,amfx1_ens3_statistics.out-unicast-pkts,amfx1_ens3_oper-status_value,amfx1_ens3_phys-address_value,...,upfx7_memory-stats.free-percent_value,upfx7_memory-stats.memory-status_value,upfx7_memory-stats.used-number_value,upfx7_memory-stats.used-percent_value,upfx7_per-core-stats.per-core-stat.idle_value,upfx7_per-core-stats.per-core-stat.io-wait_value,upfx7_per-core-stats.per-core-stat.nice_value,upfx7_per-core-stats.per-core-stat.sirq_value,upfx7_per-core-stats.per-core-stat.system_value,upfx7_per-core-stats.per-core-stat.user_value
0,15,0,0,0,1,1,1,1,0,0,...,0,0,-3308,0,-0.22,0.03,0,0.0,-0.03,0.24


In [192]:
df = pd.read_csv(r'E:\Network failure classification model using network digital twin\training-data_a.csv')

In [193]:
df.dropna(inplace=True)

In [194]:
df.head(1)

,time,source_name,y_true(fc),amfx1_ens3_oper-status,amfx1_ens3_phys-address,amfx1_ens3_speed,amfx1_ens3_statistics.in-octets,amfx1_ens3_statistics.in-unicast-pkts,amfx1_ens3_statistics.out-octets,amfx1_ens3_statistics.out-unicast-pkts,...,upfx7_memory-stats.free-percent_value,upfx7_memory-stats.memory-status_value,upfx7_memory-stats.used-number_value,upfx7_memory-stats.used-percent_value,upfx7_per-core-stats.per-core-stat.idle_value,upfx7_per-core-stats.per-core-stat.io-wait_value,upfx7_per-core-stats.per-core-stat.nice_value,upfx7_per-core-stats.per-core-stat.sirq_value,upfx7_per-core-stats.per-core-stat.system_value,upfx7_per-core-stats.per-core-stat.user_value
0,20210128175112-4_20210128175200,network-5gc-a,udmx1_vcpu-overload-start,0,0,0,1,1,1,1,...,0,0,-3308,0,-0.22,0.03,0,0.0,-0.03,0.24


In [195]:
del df['time']
del df['source_name']

In [196]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df['y_true(fc)'])
df['y_true(fc)'] = label

In [197]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [198]:
test_target = df.pop('y_true(fc)')

In [35]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.preprocessing import LabelEncoder

In [36]:
df_a = pd.read_csv(r'E:\Network failure classification model using network digital twin\training-data_a.csv')

In [37]:
df_a.head(1)

,time,source_name,y_true(fc),amfx1_ens3_oper-status,amfx1_ens3_phys-address,amfx1_ens3_speed,amfx1_ens3_statistics.in-octets,amfx1_ens3_statistics.in-unicast-pkts,amfx1_ens3_statistics.out-octets,amfx1_ens3_statistics.out-unicast-pkts,...,upfx7_memory-stats.free-percent_value,upfx7_memory-stats.memory-status_value,upfx7_memory-stats.used-number_value,upfx7_memory-stats.used-percent_value,upfx7_per-core-stats.per-core-stat.idle_value,upfx7_per-core-stats.per-core-stat.io-wait_value,upfx7_per-core-stats.per-core-stat.nice_value,upfx7_per-core-stats.per-core-stat.sirq_value,upfx7_per-core-stats.per-core-stat.system_value,upfx7_per-core-stats.per-core-stat.user_value
0,20210128175112-4_20210128175200,network-5gc-a,udmx1_vcpu-overload-start,0,0,0,1,1,1,1,...,0,0,-3308,0,-0.22,0.03,0,0.0,-0.03,0.24


In [38]:
del df_a['time']
del df_a['source_name']

In [39]:
le = LabelEncoder()
label1 = le.fit_transform(df_a['y_true(fc)'])
df_a['y_true(fc)'] = label1

In [40]:
x = df_a.drop('y_true(fc)', axis=1)
y = df_a['y_true(fc)']

In [41]:
from sklearn.metrics import recall_score, confusion_matrix, classification_report

In [42]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=2)

In [43]:
def xgb_clf(model, nrows):
    # Fit xg_reg to training set
    model.fit(X_train, y_train)
    # Predict labels of test set, y_pred
    y_pred = model.predict(X_test)
    score = recall_score(y_test, y_pred, pos_label='positive', average='micro')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    return score

In [44]:
xgb_clf(XGBClassifier(random_state=2), nrows=800)

[[ 15   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  29   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  23   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  16   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  21   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   4   0   0   0   0  12   0   0   0   0   0]
 [  0   0   0   0   0   0  22   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  26   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  16   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  18   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0   0 593   0   0   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0  13  16   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0  18   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0   0   0   0  21   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  22 

0.9692982456140351

In [45]:
xgb_clf(XGBClassifier(random_state=2), nrows=200)


[[ 15   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  29   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  23   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  16   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  21   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   4   0   0   0   0  12   0   0   0   0   0]
 [  0   0   0   0   0   0  22   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  26   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  16   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  18   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0   0 593   0   0   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0  13  16   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0  18   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0   0   0   0  21   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  22 

0.9692982456140351

In [46]:
xgb_clf(XGBClassifier(random_state=2), nrows=74)


[[ 15   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  29   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  23   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  16   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  21   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   4   0   0   0   0  12   0   0   0   0   0]
 [  0   0   0   0   0   0  22   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  26   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  16   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  18   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0   0 593   0   0   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0  13  16   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0  18   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0   0   0   0  21   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0  22 

0.9692982456140351

In [49]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [50]:
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

In [51]:
kfold = StratifiedKFold(n_splits=5)

In [52]:
base_models = []
base_models.append(('lr', LogisticRegression()))
base_models.append(('xgb', XGBClassifier()))
base_models.append(('rf', RandomForestClassifier(random_state=2)))
# define meta learner model
meta_model = LogisticRegression()
# define the stacking ensemble
clf = StackingClassifier(estimators=base_models, final_estimator=meta_model)
scores = cross_val_score(clf, x, y, cv=kfold)
print(scores.mean())

0.9755829903978052
